In [1]:
import cv2
import tensorflow as tf
import os
import numpy as np
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = 'C:\\Users\\vinayak tyagi\\Desktop\\catVsdog'
TEST_DIR = 'C:\\Users\\vinayak tyagi\\Desktop\\catVsdog-test'
IMG_SIZE = 50 
LR = 1e-3 

MODEL_NAME = 'walkvsjog-{}-{}.model'.format(LR,'2conv-basic')

In [2]:
def label_img(img):
    word_label = img.split('.')[-2]
    print(word_label)
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [4]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('_')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE , IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    np.save('test_data.npy',testing_data)
    return testing_data

In [5]:
# train_data = create_train_data()
train_data = np.load('train_data.npy')

In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
 

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [8]:
train  = train_data[:-1000]
test = train_data[1000:]

In [9]:
X = np.array([i[0] for i in train] , dtype = np.float32).reshape(-1,IMG_SIZE, IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test], dtype = np.float32).reshape(-1,IMG_SIZE, IMG_SIZE,1)
test_y = [i[1] for i in test]

In [11]:
# model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}),
# snapshot_step=50, show_metric = True, run_id = MODEL_NAME)

print("X--> ",type(X)) 
print("Y--> ",type(Y)) 
print("test_x--> ",type(X)) 
print("test_y--> ",type(X))
print(X); print(Y); print(test_x); print(test_y)
# print("MODEL--> ",model)

# model.fit(X_train, Y_train, epochs=20, batch_size=128,validation_data=(X_valid,Y_valid))

# model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), batch_size = 128)

model.fit({'input': X},{'targets': Y},
         n_epoch = 5,
         validation_set = ({'input': test_x}, {'targets': test_y}),
         batch_size = 128)

X-->  <class 'numpy.ndarray'>
Y-->  <class 'list'>
test_x-->  <class 'numpy.ndarray'>
test_y-->  <class 'numpy.ndarray'>
[[[[ 49.]
   [ 53.]
   [ 48.]
   ...
   [190.]
   [109.]
   [105.]]

  [[ 72.]
   [ 57.]
   [ 38.]
   ...
   [163.]
   [103.]
   [135.]]

  [[ 60.]
   [ 80.]
   [ 73.]
   ...
   [168.]
   [101.]
   [107.]]

  ...

  [[137.]
   [162.]
   [135.]
   ...
   [120.]
   [135.]
   [130.]]

  [[149.]
   [148.]
   [157.]
   ...
   [136.]
   [147.]
   [213.]]

  [[157.]
   [173.]
   [203.]
   ...
   [139.]
   [203.]
   [202.]]]


 [[[ 53.]
   [ 59.]
   [108.]
   ...
   [190.]
   [194.]
   [194.]]

  [[ 36.]
   [ 97.]
   [104.]
   ...
   [193.]
   [199.]
   [198.]]

  [[ 44.]
   [102.]
   [106.]
   ...
   [192.]
   [198.]
   [196.]]

  ...

  [[201.]
   [206.]
   [203.]
   ...
   [232.]
   [235.]
   [234.]]

  [[209.]
   [209.]
   [213.]
   ...
   [242.]
   [239.]
   [240.]]

  [[205.]
   [204.]
   [210.]
   ...
   [239.]
   [244.]
   [244.]]]


 [[[113.]
   [ 41.]
   [ 73.]
   

[[[[165.]
   [182.]
   [177.]
   ...
   [199.]
   [195.]
   [193.]]

  [[165.]
   [185.]
   [179.]
   ...
   [197.]
   [194.]
   [192.]]

  [[168.]
   [190.]
   [187.]
   ...
   [198.]
   [194.]
   [192.]]

  ...

  [[214.]
   [214.]
   [219.]
   ...
   [206.]
   [204.]
   [205.]]

  [[217.]
   [216.]
   [214.]
   ...
   [209.]
   [208.]
   [209.]]

  [[221.]
   [221.]
   [225.]
   ...
   [207.]
   [206.]
   [205.]]]


 [[[ 73.]
   [ 58.]
   [103.]
   ...
   [109.]
   [113.]
   [107.]]

  [[ 97.]
   [ 38.]
   [ 41.]
   ...
   [120.]
   [ 93.]
   [ 85.]]

  [[ 54.]
   [ 56.]
   [ 46.]
   ...
   [ 45.]
   [ 57.]
   [ 19.]]

  ...

  [[ 87.]
   [ 90.]
   [ 91.]
   ...
   [ 80.]
   [ 83.]
   [ 76.]]

  [[ 87.]
   [ 89.]
   [ 88.]
   ...
   [ 75.]
   [ 76.]
   [ 78.]]

  [[ 84.]
   [ 82.]
   [ 85.]
   ...
   [ 82.]
   [ 74.]
   [ 81.]]]


 [[[ 67.]
   [ 57.]
   [ 64.]
   ...
   [165.]
   [166.]
   [137.]]

  [[ 72.]
   [ 60.]
   [ 65.]
   ...
   [171.]
   [146.]
   [109.]]

  [[ 59.]
   [ 9

---------------------------------
Run id: KL7TVL
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 1023
Validation samples: 1023
--


TypeError: float() argument must be a string or a number, not 'NoneType'